# Расчет удаленности клиентов от дилерских центров

Даны два файла, содержащих идентификаторы и геопривязки (longitude, latitude) центорв продаж и всех их клиентов.
Нужно найти:
- ближайший к каждому клиенту дилерский центр;
- среднее и медиану кратчайших расстояний клиентов до дилерских центров.

Инструменты: SQL

In [1]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from pathlib import Path

В качестве СУБД используется PostgreSQL

In [51]:
with open(Path.home().joinpath('.pgpass')) as auth:
    host, port, table, user, _ = auth.read().rstrip().split(':')
    
engine = create_engine(f'postgresql+psycopg2://{user}:@{host}:{port}/{table}')

In [3]:
customers = pd.read_csv('customers_geospatial.csv', index_col=0)
customers.head()

,customer_id,longitude,latitude
0,2,-90.2625,38.5814
1,3,-87.2758,30.6143
2,4,-86.8219,36.0986
3,5,-80.4582,25.5584
4,6,-80.3187,25.6364


In [4]:
dealers = pd.read_csv('dealerships_geospatial.csv', index_col=0)
dealers.head()

,dealership_id,longitude,latitude
0,1,-74.323291,40.792846
1,2,-118.305423,34.057754
2,3,-95.307020,29.963501
3,4,-80.236454,25.801748
4,5,-122.343609,37.524487


Загрузка данных из файлов в БД

In [7]:
customers.to_sql('customers_geospatial', engine, index=False, if_exists='replace')
dealers.to_sql('dealerships_geospatial', engine, index=False, if_exists='replace')

Создадим временную таблицу с гео-точками для каждого клиента на основе широты и долготы

In [8]:
query = """
CREATE TEMP TABLE IF NOT EXISTS customer_points AS (
  SELECT
    customer_id
    , point(longitude, latitude) AS point
  FROM customers_geospatial
  WHERE longitude IS NOT NULL
  AND latitude IS NOT NULL
);
"""
engine.execute(query);

Тоже самое для каждого дилера

In [10]:
query = """
CREATE TEMP TABLE dealership_points AS (
  SELECT
    dealership_id
    , point(longitude, latitude) AS point
  FROM dealerships_geospatial
);
"""
engine.execute(query);

Рассчитаем декартово произведение расстояний между клиентами и дилерами с помощью `CROSS JOIN` и модудя `earthdistance`

In [27]:
query = """
DROP TABLE IF EXISTS customer_dealership_distance;
CREATE TEMP TABLE customer_dealership_distance AS (
  SELECT
    customer_id
    , dealership_id
    , (c.point <@> d.point)::NUMERIC * 1.609344 AS distance -- in km
    -- , ST_DistanceSphere(c.point, d.point) AS distance
  FROM customer_points c
  CROSS JOIN dealership_points d
);
"""
engine.execute(query);

Найдем кратчайшее расстояние клиент-дилер

In [28]:
query = """
DROP TABLE IF EXISTS closest_dealerships;
CREATE TEMP TABLE closest_dealerships AS (
  SELECT DISTINCT ON (customer_id)
    customer_id
    , dealership_id
    , distance
  FROM customer_dealership_distance
  ORDER BY customer_id, distance
);
"""
engine.execute(query);

`DISTINCT ON` гарантирует только одну запись для каждого уникального значения столбца в `customer_id`. В этом случае мы получим одну запись для каждого клиентского id, и, поскольку мы сортируем по расстоянию, мы получим запись с кратчайшим расстоянием.

In [29]:
distances = pd.read_sql("SELECT * FROM closest_dealerships ORDER BY customer_id;", engine)
distances

,customer_id,dealership_id,distance
34370,2,18,373.057182
37059,3,11,442.551109
32558,4,11,336.245674
11999,5,4,35.014083
7638,6,4,20.147572
...,...,...,...
23036,49995,12,151.599244
30264,49996,19,284.258485
13566,49997,15,42.175622
5642,49998,10,15.021615


Рассчитаем среднее и медианное расстояния всех клиентов до ближайших диллерских центров.

In [30]:
query = """
SELECT
  AVG(distance) AS avg_dist
  , PERCENTILE_DISC(0.5) WITHIN GROUP (ORDER BY distance) AS median_dist
FROM closest_dealerships;
"""
pd.read_sql(query, engine)

,avg_dist,median_dist
0,236.216722,146.835875


Попробуем повторить решение на python. Для измерения дистанции воспользуемся модулем `haversine`

In [14]:
import haversine as hs

In [36]:
customers = customers.assign(
    point=customers[['latitude', 'longitude']].apply(tuple, axis=1),
    key=0
)
dealers = dealers.assign(
    point=dealers[['latitude', 'longitude',]].apply(tuple, axis=1),
    key=0
)
# cross join
df = customers[['customer_id', 'point', 'key']] \
              .merge(dealers[['dealership_id', 'point', 'key']],
                     on='key',
                     how='outer',
                     suffixes=['_c', '_d']) \
              .assign(distance=lambda df: df[['point_c', 'point_d']] \
                      .apply(lambda x: hs.haversine(*x.values), axis=1))

In [37]:
closest_dealerships = pd.crosstab(
    df['dealership_id'],
    df['customer_id'],
    df['distance'],
    aggfunc='min'
).idxmin().reset_index(name='dealership_id') \
          .merge(df[['customer_id', 'dealership_id', 'distance']])

closest_dealerships

,customer_id,dealership_id,distance
0,2,18,373.058465
1,3,11,442.552632
2,4,11,336.246830
3,5,4,35.014203
4,6,4,20.147641
...,...,...,...
44528,49995,12,151.599766
44529,49996,19,284.259462
44530,49997,15,42.175767
44531,49998,10,15.021667


In [38]:
closest_dealerships['distance'].describe().loc[['mean', '50%']]

mean    236.217534
50%     146.836380
Name: distance, dtype: float64